In [6]:
%run rf_real_vs_synthetic.ipynb
'y_real' in globals(), 'X_real' in globals()

(91, 63)
y    68
dtype: int64


### First 5 rows of synthetic data (HIV-)

### Last 5 rows of synthetic data (HIV+)

Sampled indices: [15  9 13  1 22 87 29 57 74 72]


### AUC (one iteration): 0.9133

...

### AUC (one iteration): 0.9578

Train shape: (80, 63)
Test shape: (30, 63)
Train real count: 40
Train syn count: 40
Test real count: 15
Test syn count: 15
Mean real (test): 431.80017321829513
Mean syn  (test): 368.2515972476541
Std real: 837.2267509517243
Std syn : 640.1838440519552


,feature,mean_diff
26,V10_blood_IgGRBD,-481.4
25,V9_blood_IgGRBD,-422.7
58,V9Neut,-233.2
16,V10_blood_IgGspike,-227.4
21,V6_blood_IgGRBD,216.0
27,V11_blood_IgGRBD,176.6
62,V9_ACE2,-138.4
57,V8Neut,120.8
24,V8b_blood_IgGRBD,76.6
23,V8a_blood_IgGRBD,-73.3


\+ is favour synthetic, - is favour real

(True, True)

In [7]:

real_neg = np.where(y_real == 0)[0]
real_pos = np.where(y_real == 1)[0]
syn_neg  = np.where(y_syn  == 0)[0]
syn_pos  = np.where(y_syn  == 1)[0]

#test/holdout: 3:12 
test_real_idx = strat_samp(real_neg, real_pos, 3, 12)
test_syn_idx  = strat_samp(syn_neg,  syn_pos,  3, 12)

# remaining indices for training 
rem_real = np.setdiff1d(np.arange(X_real.shape[0]), test_real_idx)
rem_syn  = np.setdiff1d(np.arange(X_syn.shape[0]),  test_syn_idx)

rem_real_neg = rem_real[y_real[rem_real] == 0]
rem_real_pos = rem_real[y_real[rem_real] == 1]
rem_syn_neg  = rem_syn[y_syn[rem_syn] == 0]
rem_syn_pos  = rem_syn[y_syn[rem_syn] == 1]


train_real_idx = strat_samp(rem_real_neg, rem_real_pos, 20, 20)
train_syn_idx  = strat_samp(rem_syn_neg,  rem_syn_pos,  20,  20)


# Real vs. Syn -classification dataset prep
X_train = np.vstack([X_real[train_real_idx], X_syn[train_syn_idx]])
s_train = np.concatenate([np.zeros(len(train_real_idx), dtype=int),
                            np.ones(len(train_syn_idx), dtype=int)])

X_test  = np.vstack([X_real[test_real_idx],  X_syn[test_syn_idx]])
s_test  = np.concatenate([np.zeros(len(test_real_idx), dtype=int),
                            np.ones(len(test_syn_idx), dtype=int)])


rf = train_rf(X_train, s_train)

p_syn = rf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(s_test, p_syn)


display(Markdown(f"### AUC (one iteration): {auc:.4f}"))
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

print("Train real count:", (s_train == 0).sum())
print("Train syn count:", (s_train == 1).sum())

print("Test real count:", (s_test == 0).sum())
print("Test syn count:", (s_test == 1).sum())

print("Mean real (test):", X_test[s_test==0].mean())
print("Mean syn  (test):", X_test[s_test==1].mean())
print("Std real:", X_test[s_test==0].std())
print("Std syn :", X_test[s_test==1].std())

### AUC (one iteration): 1.0000

Train shape: (80, 63)
Test shape: (30, 63)
Train real count: 40
Train syn count: 40
Test real count: 15
Test syn count: 15
Mean real (test): 423.560051550207
Mean syn  (test): 361.62321246656535
Std real: 816.6258011524069
Std syn : 627.9571373572553


# Feature-wise means

In [8]:
mean_diff = X_syn.mean(axis=0) - X_real.mean(axis=0)

df_diff = pd.DataFrame({
    "feature": col_names,
    "mean_diff": mean_diff
})

df_diff_sorted = df_diff.reindex(
    df_diff["mean_diff"].abs().sort_values(ascending=False).index
)
df_diff_sorted["mean_diff"] = df_diff["mean_diff"].round(1)

display(df_diff_sorted.head(25)) 
display(Markdown("\\+ is favour synthetic, - is favour real"))


,feature,mean_diff
26,V10_blood_IgGRBD,-481.4
25,V9_blood_IgGRBD,-422.7
58,V9Neut,-233.2
16,V10_blood_IgGspike,-227.4
21,V6_blood_IgGRBD,216.0
27,V11_blood_IgGRBD,176.6
62,V9_ACE2,-138.4
57,V8Neut,120.8
24,V8b_blood_IgGRBD,76.6
23,V8a_blood_IgGRBD,-73.3


\+ is favour synthetic, - is favour real